In [34]:
import json
import pandas as pd

In [2]:
def load_jsonl(input_path) -> list:
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [12]:
# load jsonl file to list of dictionaries
comments_path = './data/comments_from_sample.jsonl'
posts_path = './data/6000_posts_sample.jsonl'
comments_6000 = load_jsonl(comments_path)
posts_6000 = load_jsonl(posts_path)

Loaded 3870 records from ./data/comments_from_sample.jsonl
Loaded 6000 records from ./data/6000_posts_sample.jsonl


In [62]:
comments_6000[0].keys()

dict_keys(['owner', 'score', 'post_type', 'creation_date', 'post_id', 'comment_id', 'body_markdown', 'body'])

In [63]:
posts_6000[0].keys()

dict_keys(['tags', 'is_answered', 'view_count', 'accepted_answer_id', 'answer_count', 'score', 'last_activity_date', 'creation_date', 'question_id', 'content_license', 'link', 'title', 'body', 'answers', 'upvotes', 'comments'])

In [52]:
i = 0
for post in posts_6000:
    for comment in comments_6000:
        if post.get('question_id') == comment.get('post_id'):
            i += 1
print(i)

1


In [60]:
def find_outdated_in_comments(data):
    outdated_comments = pd.DataFrame(columns = ['Id', 'Text', 'PostId', 'PostLink'])
    id_column = []
    text_column = []
    postid_column = []
    postlink_column = []
    for comment in data:
            if ('outdated' in comment['body'] or 'deprecated' in comment['body']
                or 'obsolete' in comment['body'] or 'out of date' in comment['body']):
                id_column.append(comment['comment_id'])
                text_column.append(comment['body'])
                postid_column.append(comment['post_id'])
                postlink_column.append(comment['owner']['link'])
                #print('CommentId: ', comment['Id'])
                #print('Text: ', comment['Text'])
                #print('PostId: ', comment['PostId'])
    outdated_comments['Id'] = id_column
    outdated_comments['Text'] = text_column
    outdated_comments['PostId'] = postid_column
    outdated_comments['PostLink'] = postlink_column
    return outdated_comments

In [61]:
find_outdated_in_comments(comments_6000)

,Id,Text,PostId,PostLink
0,62979626,I got this error from php: <code>Automatically...,37754566,https://stackoverflow.com/users/1609729/oto-sh...
1,54586293,"According to <a href=""http://stackoverflow.com...",33393350,https://stackoverflow.com/users/3740093/visual...
2,57532771,I want to detect deprecated IPv6 using C code....,34896415,https://stackoverflow.com/users/5510990/neel
3,50434298,"In 3.5.1, &quot;Devise.bcrypt is deprecated; u...",31216689,https://stackoverflow.com/users/720164/jonatha...
4,62331056,"Not an answer to your question, but your code ...",37411386,https://stackoverflow.com/users/766786/amedee-...
5,47502218,He is not using parse(String) but parse(format...,26676586,https://stackoverflow.com/users/1997707/john-m...
6,58838806,"&#39;gets()&#39; is deprecated, use fgets() in...",35576152,https://stackoverflow.com/users/1582263/tigris
7,58781821,@deceze: Answer changed to focus on the &quot;...,35536599,https://stackoverflow.com/users/169224/kristof...


In [70]:
def find_outdated_in_answers(data):
    outdated_answers = pd.DataFrame(columns = ['Id', 'Body', 'PostId'])
    id_column = []
    body_column = []
    postid_column = []
    postlink_column = []
    for post in data:
        if post['answer_count'] != 0:
            answers = post['answers']
            answerKeys = list(answers.keys())
            for i in range(len(answerKeys)):
                answer = answers[answerKeys[i]]
                body = answer['body']
                if ('outdated' in body or 'deprecated' in body or 'obsolete' in body or 'out of date' in body):
                    id_column.append(answer['answer_id'])
                    body_column.append(answer['body'])
                    postid_column.append(answer['question_id'])
                    postlink_column.append(post['link'])
                    #print('AnswerId: ', answer['Id'])
                    #print('Body: ', answer['Body'])
                    #print('PostId: ', answer['ParentId'])
    outdated_answers['Id'] = id_column
    outdated_answers['Body'] = body_column
    outdated_answers['PostId'] = postid_column
    outdated_answers['PostLink'] = postlink_column
    return outdated_answers

In [71]:
find_outdated_in_answers(posts_6000)

,Id,Body,PostId,PostLink
0,60823374,"<p>The <a href=""https://www.npmjs.com/package/...",60821313,https://stackoverflow.com/questions/60821313/s...
1,44163299,"<p>According to the docs <a href=""http://docs....",44159204,https://stackoverflow.com/questions/44159204/a...
2,56126720,<p>Step function seems the best solution here....,44668605,https://stackoverflow.com/questions/44668605/a...
3,55761393,<p>This what The python 3.7 AWS lambda environ...,55295305,https://stackoverflow.com/questions/55295305/a...
4,43285441,<p><strong>These answers are out of date.</str...,21917661,https://stackoverflow.com/questions/21917661/c...
5,59142446,<p>This is a general question about storage te...,59141973,https://stackoverflow.com/questions/59141973/i...
6,43316867,<p>I had the same issue. The managed policies ...,28906981,https://stackoverflow.com/questions/28906981/a...
7,52563919,<p>The error message refers to <code>part_numb...,52495435,https://stackoverflow.com/questions/52495435/a...
8,46718013,<p>The issue is that when you use the old cons...,46697687,https://stackoverflow.com/questions/46697687/k...
9,52899085,<p>ElastiCache docs are way out of date; new a...,52824961,https://stackoverflow.com/questions/52824961/c...
